In [1]:
from torchaudio.models.decoder import ctc_decoder
import numpy as np 
import torch
from torch.nn.functional import softmax, log_softmax
from brainaudio.inference.eval_metrics import _cer_and_wer
from brainaudio.inference.lm_funcs import normalize_shorthand, compute_wer_from_logits
import pandas as pd

In [2]:
language_model_path = "/data2/brain2text/lm/"
units_txt_file_pytorch = f"{language_model_path}units_pytorch.txt"
imagineville_vocab_phoneme = f"{language_model_path}vocab_lower_100k_pytorch_phoneme.txt"

In [3]:
decoder = ctc_decoder(tokens=units_txt_file_pytorch, lexicon=imagineville_vocab_phoneme, 
                       beam_size=300, nbest=20, lm="/data2/brain2text/lm/lm_dec19_huge_4gram.kenlm", 
                       lm_weight=2.0, word_score=0.1, log_add=True, sil_score=0, beam_threshold=1e3)

[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached limit: 6
[Trie] Trie label number reached l

In [4]:
logits_paths = [
    "/data2/brain2text/b2t_25/logits/baseline_hpo_combined_trial_12/logits_val_chunk_1_context_250.npz", 
    "/data2/brain2text/b2t_25/logits/baseline_hpo_combined_trial_12/logits_val_chunk_5_context_50.npz",
    "/data2/brain2text/b2t_25/logits/baseline_hpo_combined_trial_12/logits_val_chunk_10_context_25.npz",
    "/data2/brain2text/b2t_25/logits/baseline_hpo_combined_trial_12/logits_val_chunk_full_context_full.npz"
]

dataset_ids = [25, 25, 25, 25]  # Specify 24 or 25 for each logits file

In [5]:

wer_dict, wer_list = compute_wer_from_logits(
    logits_paths=logits_paths,
    dataset_ids=dataset_ids,
    decoder=decoder,
    acoustic_scale=0.6,
    verbose=True,
    save_results=True,
    output_filename="wer_results.pkl"
)

print("\nWER Results:")
for key, wer in wer_dict.items():
    print(f"  {key}: {wer:.4f}")


Processing: /data2/brain2text/b2t_25/logits/baseline_hpo_combined_trial_12/logits_val_chunk_1_context_250.npz
  Dataset: b2t_25
  Trial 0/1426
  Trial 100/1426
  Trial 100/1426
  Trial 200/1426
  Trial 200/1426
  Trial 300/1426
  Trial 300/1426
  Trial 400/1426
  Trial 400/1426
  Trial 500/1426
  Trial 500/1426
  Trial 600/1426
  Trial 600/1426
  Trial 700/1426
  Trial 700/1426
  Trial 800/1426
  Trial 800/1426
  Trial 900/1426
  Trial 900/1426
  Trial 1000/1426
  Trial 1000/1426
  Trial 1100/1426
  Trial 1100/1426
  Trial 1200/1426
  Trial 1200/1426
  Trial 1300/1426
  Trial 1300/1426
  Trial 1400/1426
  Trial 1400/1426
  WER: 0.0872

Processing: /data2/brain2text/b2t_25/logits/baseline_hpo_combined_trial_12/logits_val_chunk_5_context_50.npz
  Dataset: b2t_25
  Trial 0/1426
  WER: 0.0872

Processing: /data2/brain2text/b2t_25/logits/baseline_hpo_combined_trial_12/logits_val_chunk_5_context_50.npz
  Dataset: b2t_25
  Trial 0/1426
  Trial 100/1426
  Trial 100/1426
  Trial 200/1426
  Tri

In [8]:
wer_results = pd.read_pickle("/data2/brain2text/b2t_25/logits/baseline_hpo_combined_trial_12/wer_results.pkl")

print("WER Results:")
print("-" * 60)
for key, value in wer_results.items():
    print(f"{key:50s} {value:.4f}")
print("-" * 60)

WER Results:
------------------------------------------------------------
logits_val_chunk_1_context_250                     0.0872
logits_val_chunk_5_context_50                      0.0839
logits_val_chunk_10_context_25                     0.0836
logits_val_chunk_full_context_full                 0.0819
------------------------------------------------------------
